# Factors.jl
A package for factors/potentials with parametric supports

The core of `Factors.jl` runs on [`Dimensions`](#Dimensions)

A factor maps from the support of each of its dimensions to a `Float64`.

Potentials are read in column-major order: the first dimension corresponds to the first axis (column), the second dimension corresponds to the second axis (rows) etc ...

In [2]:
ft = Factor([:X, :Y], [1 4; 2 5; 3 6])
DataFrame(ft)

LoadError: UndefVarError: Factor not defined

There are multiple convience constructors:

In [ ]:
c = Dimension(:C, 3)
s = Dimension(:S, 10:2:18)

Factor([c, s], rand(3, 5))
Factor(c, [2, 0, 16])
Factor([1 4; 2 5; 3 6], :X => 3:5, :Y => ['a', 'b'])

# dimension `i` goes from `1` to `size(potential, i)`
Factor([:X, :Y], rand(20, 16)) 
Factor(:X, [31, 33, 58])

# potentials with uniform values
Factor(c)  # all zeros
Factor(c, 31) # all 31's
Factor([c, s], nothing) # unitialized
Factor([c, s], 16)
Factor(Dict(:X=>14, :Y=>['Γ', 'Δ'], :Z =>'a':2:'z'))
Factor(Dict(:X=>14, :Y=>['Γ', 'Δ'], :Z =>'a':2:'z'), nothing)
Factor(:A=>10, :B=>3:20)
Factor(16, :A=>10, :B=>3:20) # all 16

# zero dimensional factor
Factor(2016)

A Factors scope is its dimensions:

In [15]:
scope(ft)

2-element Array{Factors.Dimension,1}:
 C:  1:3          
 S:  'a':2:'g' (4)

In [ ]:
names(ft)

### As an Array

In [ ]:
similar(ft) # unitialized potential

In [ ]:
size(ft, :A, :S)

In [ ]:
length(ft)

In [ ]:
ndims(ft)

### Iterating

Iterating over a factor gives the values in the potential

Iterating over assignments is also possible

### Indexing

An `Assignment` (or `Pair`s) can also select from the factor:

In [ ]:
ft = Factor([1 4; 2 5; 3 6], :X => 2:4, :Y=>['a', 'b'])
DataFrame(ft)

In [ ]:
ft[:X=>[3, 2], :Y=>'a']

In [ ]:
ft[Assignment(:X=>[3, 2], :Y=>'a')] = [20, 16]
DataFrame(ft)

Besides also overloading `sub2ind` and `ind2sub`, functions to convert from and `Assignment`s are provided:

In [ ]:
at2sub(ft, 3, 'b')

In [ ]:
a2sub(ft, Assignment(:Y=>'a', :X=>4))

In [ ]:
sub2a(ft, 2, 2)

In [ ]:
sub2at(ft, 2, 2)

### Iterating

Iterating over a factor returns assignments as tuples

In [ ]:
ft = Factor(:X=>'a':'c', :Y=>2:4, :Z=>1:3)
for t in ft
    println(t)
end

### Patterns

`pattern` returns the sequence of a dimension in a factor

In [13]:
c = Dimension(:C, 3)
s = Dimension(:S, 'a':2:'h')
ft = Factor([c, s])

pattern(ft)

12×2 Array{Int64,2}:
 1  1
 2  1
 3  1
 1  2
 2  2
 3  2
 1  3
 2  3
 3  3
 1  4
 2  4
 3  4

Also done via accessing that dimension

In [14]:
pattern_states(ft, :A)

12×2 Array{Any,2}:
 1  'a'
 2  'a'
 3  'a'
 1  'c'
 2  'c'
 3  'c'
 1  'e'
 2  'e'
 3  'e'
 1  'g'
 2  'g'
 3  'g'

### Mapping

In [ ]:
rand(ft)

### Broadcasting

Operations can be broadcast along dimensions:

DataFrame(broadcast(*, ft, :B, [100, 0.01]))

Dimensions can be reduced.
Convience functions are provded for the following (As well as for their excited cousins, e.g. sum!):
* `sum`  
* `prod`  
* `maximum`  
* `minimum`  

### Joining

Factors can be joined through `join` or by multiplying (adding, etc.) them:

### Negatives

By default, negatives are allowed in factors:

This can be changed to raise a warning or to throw an error

In [ ]:
set_negative_mode(NegativeWarn)
set_negative_mode(NegativeError)
set_negative_mode(NegativeIgnore)

## Dimensions

The core unit are dimensions, which are names (`Symbol`) with countably-finite supports (`<: AbstractVector`):

In [13]:
ds = map(s -> Dimension(:X, s), [["bob", "waldo", "superman"], ('a', 'α'), 'a':2:'z', 10:3:40, 2:15, 1:4, 16, []])

8-element Array{Any,1}:
 X:  String["bob","waldo","superman"] (3)
 X:  ['a','α'] (2)                       
 X:  'a':2:'y' (13)                      
 X:  10:3:40 (11)                        
 X:  2:15 (14)                           
 X:  1:4                                 
 X:  1:16                                
 X:  Any[] (0)                           

In [14]:
map(eltype, ds)

8-element Array{Any,1}:
 String
 Char  
 Char  
 Int64 
 Int64 
 Int64 
 Int64 
 Any   

In [15]:
map(spttype, ds)

8-element Array{Any,1}:
 Array{String,1}       
 Array{Char,1}         
 StepRange{Char,Int64} 
 StepRange{Int64,Int64}
 UnitRange{Int64}      
 Base.OneTo{Int64}     
 Base.OneTo{Int64}     
 Array{Any,1}          

### Indexing and iterating

In [16]:
x = Dimension(:X, 'α':'ω')

for v in x
    print(v, " ")
end

α β γ δ ε ζ η θ ι κ λ μ ν ξ ο π ρ ς σ τ υ φ χ ψ ω 

In [17]:
x[2]

'β'

In [18]:
indexin('β', x)

2

In [19]:
(i, d) = update(x, ['α', 'ψ', 'ζ', 'δ'])

([1,24,6,4],X:  ['α','ψ','ζ','δ'] (4))

### Dimension Comparisons

Equality for dimensions is by state values, not by type

In [8]:
Dimension(:X, [1, 2, 3]) == Dimension(:X, 1:1:3)  == Dimension(:X, 1:3) == Dimension(:X, 3)

true

Comparisons use the position of elements in a dimension

In [9]:
o = Dimension(:X, [3, 16, -2])
o .< -2 # here, 3 & 16 are less than -2

3-element BitArray{1}:
  true
  true
 false

In [10]:
# 3, 16, and -2 are all ≥ 3
o .≥ 3

3-element BitArray{1}:
 true
 true
 true